In [ ]:
"""
Tool that allows the user to assign cameras to stretches of the 360 footage.

"""
from pathlib import Path
import cv2

camera_folder = Path()

cameras = []

for camera in camera_folder.iterdir():
    if camera.is_dir():
        for file in camera.iterdir():
            if file.suffix == '.mp4':
                cameras.append(file)
        print(camera.name)
        if '360' in camera.name:
            omni_cam = camera
            assert cv2.VideoCapture(camera).get(cv2.CAP_PROP_FRAME_COUNT) > 0, "Omni video file is empty"
        else:
            cameras.append(camera)


assert len(cameras) > 0, "There should be at least one camera"
assert omni_cam is not None, "There should be a 360 camera"

fps_360 = 30
skip_360 = fps_360 // 2
length_360 = 
segment_length = 30

# by default each reconstruction will be made with 30 frames, more frames will cause the reconstruction to take longer
num_segments = (length_360 / skip_360) // segment_length

# load a frame from each camera
base_frames = []
for camera in cameras:
    cap = cv2.VideoCapture(str(camera))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, frame = cap.read()
    assert ret, "Camera {} is empty".format(camera.name)
    base_frames.append(frame)




In [1]:
import matplotlib.pyplot as plt

import numpy as np

def find_factors(a):
    root = np.sqrt(a)
    b = int(np.ceil(root))
    c = int(np.floor(root))
    
    # If b*c is less than a, increment b
    while b * c < a:
        b += 1

    return b, c

omni_frames = []
for i in range(0, 30 * 30, 30):
    omni_frames.append(omni_cam.get_frame(i))

num_cameras = len(cameras) + 1

fig, axs = plt.subplots((find_factors(num_cameras)), figsize=(20, 20))


for i, ax in enumerate(axs):
    if i == 0:
        continue
    ax.imshow(base_frames[i])
    ax.set_title(f'Camera {i}')

axs[0].imshow(omni_frames[0])

plt.show()

NameError: name 'omni_cam' is not defined